In [1]:
from pathlib import Path
from os.path import expanduser
from os import path
import numpy as np
import sys
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import models
from torchsummary import summary
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, datasets
import os

In [2]:
max_dataset_size = 1000000;
waveform_length = 72;
nb_of_elements = 150000;
nb_of_elements = 150;

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
nb_of_datasets = 45;
snr_from = 20;
snr_to = 100;
max_shift = 18;
use_horizontal_flip = False;
use_vertical_flip = False;
filter_type = 'high';
sampling_rate = 24000;
passband = np.array([100], dtype=int);
order = 1;



In [3]:
root_folder = os.path.dirname(os.getcwd())
path_to_recordings = path.join(root_folder,'data/synthesized');
path_to_ground_truth_data = path.join(root_folder,'data/synthesized');
path_to_train_data = path.join(root_folder,'data/detect_train.npy');
path_to_train_labels = path.join(root_folder,'data/detect_labels.npy');
path_to_noise_data = path.join(root_folder,'data/noise_data.npy');
path_to_train_data1 = path.join(root_folder,'data/detect_train1.npy');
path_to_train_labels1 = path.join(root_folder,'data/detect_labels1.npy');

sys.path.append(root_folder)
from custom_resnet import CustomResnet as cnn
transform_list =[cnn.FilterSignalUsingButtersWorth(filter_type, sampling_rate, passband, order), cnn.OptimizedZScoreNormalizaton()];



In [4]:

# counts each class
classes_per_recording = np.zeros((nb_of_datasets), dtype='int')
for i in range(0 , nb_of_datasets):
    single_recording_ground_truth = path.join(path_to_ground_truth_data, 'gt_' + str(i + 1) + '.npy');
    gt_data = np.load(single_recording_ground_truth);
    classes_per_recording[i] = np.unique(gt_data[1,:]).size;
    print(np.unique(gt_data[1,:]))
    print(np.min(gt_data))
print(classes_per_recording)

# class counter to concatenate classes together from different recordings
class_counter = np.zeros((nb_of_datasets, 1), dtype='int');
class_counter[0] = 0;
for i in range(1, nb_of_datasets):
    class_counter[i] = class_counter[i-1] + classes_per_recording[i - 1] - 1; 
print(class_counter)



[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16.]
0.0
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19.]
0.0
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12.]
0.0
[0. 1. 2. 3. 4.]
0.0
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15.]
0.0
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11.]
0.0
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.]
0.0
[0. 1. 2.]
0.0
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19.]
0.0
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19. 20.]
0.0
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19. 20.]
0.0
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19. 20.]
0.0
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10.]
0.0
[0. 1. 2. 3.]
0.0
[0. 1. 2. 3. 4. 5. 6. 7. 8. 9.]
0.0
[0. 1. 2. 3. 4. 5. 6. 7. 8.]
0.0
[0. 1. 2. 3. 4. 5. 6. 7. 8. 9.]
0.0
[0. 1. 2. 

In [5]:
# couts sample count for each class
nb_of_classes = int(np.sum(classes_per_recording - 1) + 1);


In [6]:
print(classes_per_recording)
print(class_counter)

[17 20 13  5 16 12 18  3 20 21 21 21 11  4 10  9 10  8 19 15  5  7 18  7
 17 14  6 15  4  6 12 11  5  9 13 19 11 14  3  8 16 10 17 14  9]
[[  0]
 [ 16]
 [ 35]
 [ 47]
 [ 51]
 [ 66]
 [ 77]
 [ 94]
 [ 96]
 [115]
 [135]
 [155]
 [175]
 [185]
 [188]
 [197]
 [205]
 [214]
 [221]
 [239]
 [253]
 [257]
 [263]
 [280]
 [286]
 [302]
 [315]
 [320]
 [334]
 [337]
 [342]
 [353]
 [363]
 [367]
 [375]
 [387]
 [405]
 [415]
 [428]
 [430]
 [437]
 [452]
 [461]
 [477]
 [490]]


In [7]:
16+19+12

47

In [8]:
max_dataset_size = max_dataset_size + nb_of_classes - max_dataset_size % nb_of_classes;
samples_per_class = max_dataset_size // nb_of_classes;
print(samples_per_class)
print(max_dataset_size / nb_of_classes)
print(max_dataset_size)


2005
2005.0
1000495


In [9]:
unique_list = [];
dataset_spikes = np.array([])
labels_spikes = np.array([], dtype = "int")
# adds single unit activity data
for i in range(0, nb_of_datasets):
    added_sample_counter = np.zeros((classes_per_recording[i] - 1));
    data_iter_counter = 0;
    total_nb_of_samples = int((classes_per_recording[i] - 1) * samples_per_class);
    temp_spikes = np.zeros((total_nb_of_samples, 1, waveform_length));
    temp_labels = np.zeros((1, total_nb_of_samples), dtype='int');
    print(dataset_spikes.shape)
    # generates augmentations until there are same sized samples for each class
    while (data_iter_counter < total_nb_of_samples):
        single_recording_path = path.join(path_to_recordings, 'data_' + str(i + 1) + '.npy');
        single_recording_ground_truth = path.join(path_to_ground_truth_data, 'gt_' + str(i + 1) + '.npy');
        generated_spikes = cnn.GenerateDataset(single_recording_path, single_recording_ground_truth, waveform_length, 600, snr_from, snr_to, max_shift, use_horizontal_flip, use_vertical_flip, transform_list);
        # adds to temp variable to add to real dataset afterwards
        for j, (data, target) in enumerate(generated_spikes):
            neuron_index = target.item();
            if(neuron_index > 0 and added_sample_counter[neuron_index - 1] < samples_per_class):
                spike_waveform =  (data.numpy());
                temp_spikes[data_iter_counter, :] = spike_waveform;
                class_index = class_counter[i] + neuron_index;
                # changes class index
                temp_labels[0, data_iter_counter] = class_index;
                added_sample_counter[neuron_index - 1] = added_sample_counter[neuron_index - 1] + 1;
                data_iter_counter = data_iter_counter + 1;
    unique_list.append(np.unique(temp_labels));
    print(added_sample_counter)
    if (dataset_spikes.size == 0):
        dataset_spikes = temp_spikes;
    else:
        dataset_spikes = np.concatenate((dataset_spikes, temp_spikes), axis=0)  
        
    if (labels_spikes.size == 0): 
        labels_spikes = temp_labels;
    else:
        labels_spikes = np.concatenate((labels_spikes, temp_labels), axis=1)            


(0,)
========== 1 generation ==========
temp_dataset_len:  0
shift_from:  -18
shift_to:  19
shift_step:  1
shift_indexes:  tensor([-18, -17, -16, -15, -14, -13, -12, -11, -10,  -9,  -8,  -7,  -6,  -5,
         -4,  -3,  -2,  -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
         10,  11,  12,  13,  14,  15,  16,  17,  18], dtype=torch.int32)
snr_ratio:  87.71806086556711
flip_data_horz:  0
[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7feef4701860>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7fef0460f550>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7fef0460f518>]
tensor(16, dtype=torch.int32)


/home/vtpc/Documents/Alvils/spike-sorting/custom_resnet/CustomResnet.py:293: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than tensor.new_tensor(sourceTensor).
  temp = temp.new_tensor(data);


dataset len:  444296
[2005. 2005. 2005. 2005. 2005. 2005. 2005. 2005. 2005. 2005. 2005. 2005.
 2005. 2005. 2005. 2005.]
(32080, 1, 72)
========== 1 generation ==========
temp_dataset_len:  0
shift_from:  -18
shift_to:  19
shift_step:  1
shift_indexes:  tensor([-18, -17, -16, -15, -14, -13, -12, -11, -10,  -9,  -8,  -7,  -6,  -5,
         -4,  -3,  -2,  -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
         10,  11,  12,  13,  14,  15,  16,  17,  18], dtype=torch.int32)
snr_ratio:  22.68003854455385
flip_data_horz:  0
[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7feef4701940>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7fef0460f550>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7fef0460f518>]
tensor(19, dtype=torch.int32)
dataset len:  472786
========== 1 generation ==========
temp_dataset_len:  0
shift_from:  -18
shift_to:  19
shift_step:  1
shift_indexes:  tensor([-18, -17, -

[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7feef4701908>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7fef0460f550>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7fef0460f518>]
tensor(20, dtype=torch.int32)
dataset len:  554075
[2005. 2005. 2005. 2005. 2005. 2005. 2005. 2005. 2005. 2005. 2005. 2005.
 2005. 2005. 2005. 2005. 2005. 2005. 2005. 2005.]
(310775, 1, 72)
========== 1 generation ==========
temp_dataset_len:  0
shift_from:  -18
shift_to:  19
shift_step:  1
shift_indexes:  tensor([-18, -17, -16, -15, -14, -13, -12, -11, -10,  -9,  -8,  -7,  -6,  -5,
         -4,  -3,  -2,  -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
         10,  11,  12,  13,  14,  15,  16,  17,  18], dtype=torch.int32)
snr_ratio:  49.25765785991695
flip_data_horz:  0
[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7feef47017b8>, <custom_resnet.Custo

[<custom_resnet.CustomResnet.Awgn object at 0x7feef47017b8>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7fef0460f550>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7fef0460f518>]
tensor(4, dtype=torch.int32)
dataset len:  158693
[2005. 2005. 2005. 2005.]
(515285, 1, 72)
========== 1 generation ==========
temp_dataset_len:  0
shift_from:  -18
shift_to:  19
shift_step:  1
shift_indexes:  tensor([-18, -17, -16, -15, -14, -13, -12, -11, -10,  -9,  -8,  -7,  -6,  -5,
         -4,  -3,  -2,  -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
         10,  11,  12,  13,  14,  15,  16,  17,  18], dtype=torch.int32)
snr_ratio:  43.63348273969867
flip_data_horz:  0
[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7feef4701668>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7fef0460f550>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7fef0460f518>]
te

dataset len:  304214
[2005. 2005. 2005. 2005. 2005. 2005. 2005. 2005. 2005. 2005. 2005.]
(707765, 1, 72)
========== 1 generation ==========
temp_dataset_len:  0
shift_from:  -18
shift_to:  19
shift_step:  1
shift_indexes:  tensor([-18, -17, -16, -15, -14, -13, -12, -11, -10,  -9,  -8,  -7,  -6,  -5,
         -4,  -3,  -2,  -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
         10,  11,  12,  13,  14,  15,  16,  17,  18], dtype=torch.int32)
snr_ratio:  23.927318220410637
flip_data_horz:  0
[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7feef0760b70>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7fef0460f550>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7fef0460f518>]
tensor(10, dtype=torch.int32)
dataset len:  335775
[2005. 2005. 2005. 2005. 2005. 2005. 2005. 2005. 2005. 2005.]
(727815, 1, 72)
========== 1 generation ==========
temp_dataset_len:  0
shift_from:  -18
shift_to:  19
s

[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7feef0760ba8>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7fef0460f550>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7fef0460f518>]
tensor(15, dtype=torch.int32)
dataset len:  493728
[2005. 2005. 2005. 2005. 2005. 2005. 2005. 2005. 2005. 2005. 2005. 2005.
 2005. 2005. 2005.]
(906260, 1, 72)
========== 1 generation ==========
temp_dataset_len:  0
shift_from:  -18
shift_to:  19
shift_step:  1
shift_indexes:  tensor([-18, -17, -16, -15, -14, -13, -12, -11, -10,  -9,  -8,  -7,  -6,  -5,
         -4,  -3,  -2,  -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
         10,  11,  12,  13,  14,  15,  16,  17,  18], dtype=torch.int32)
snr_ratio:  29.30779664227029
flip_data_horz:  0
[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7feef4701860>, <custom_resnet.CustomResnet.FilterSignalUsingButte

In [10]:

dataset_multiunit_spikes = np.array([])
labels_multiunit_spikes = np.array([], dtype = "int")
    
# adds multi unit activity data
data_iter_counter = 0;
total_nb_of_samples = int(labels_spikes.size);
temp_spikes = np.zeros((total_nb_of_samples, 1, waveform_length));
temp_labels = np.zeros((1, total_nb_of_samples), dtype='int');
while (data_iter_counter < total_nb_of_samples):
    for i in range(0, nb_of_datasets):
        single_recording_path = path.join(path_to_recordings, 'data_' + str(i + 1) + '.npy');
        single_recording_ground_truth = path.join(path_to_ground_truth_data, 'gt_' + str(i + 1) + '.npy');
        generated_spikes = cnn.GenerateDataset(single_recording_path, single_recording_ground_truth, waveform_length, 600, snr_from, snr_to, max_shift, use_horizontal_flip, use_vertical_flip, transform_list);
        # adds to temp variable to add to real dataset afterwards
        for j, (data, target) in enumerate(generated_spikes):
            neuron_index = target.item();
            if(neuron_index == 0 and data_iter_counter < total_nb_of_samples):
                spike_waveform =  (data.numpy());
                temp_spikes[data_iter_counter, :] = spike_waveform;
                class_index = neuron_index;
                # changes class index
                temp_labels[0, data_iter_counter] = class_index;
                data_iter_counter = data_iter_counter + 1;
                
print(added_sample_counter)
if (dataset_multiunit_spikes.size == 0):
    dataset_multiunit_spikes = temp_spikes;
else:
    dataset_multiunit_spikes = np.concatenate((dataset_multiunit_spikes, temp_spikes), axis=0)  
        
if (labels_multiunit_spikes.size == 0): 
    labels_multiunit_spikes = temp_labels;
else:
    labels_multiunit_spikes = np.concatenate((labels_multiunit_spikes, temp_labels), axis=1)          

========== 1 generation ==========
temp_dataset_len:  0
shift_from:  -18
shift_to:  19
shift_step:  1
shift_indexes:  tensor([-18, -17, -16, -15, -14, -13, -12, -11, -10,  -9,  -8,  -7,  -6,  -5,
         -4,  -3,  -2,  -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
         10,  11,  12,  13,  14,  15,  16,  17,  18], dtype=torch.int32)
snr_ratio:  76.6123711442678
flip_data_horz:  0
[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7feef47088d0>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7fef0460f550>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7fef0460f518>]
tensor(16, dtype=torch.int32)
dataset len:  444296
========== 1 generation ==========
temp_dataset_len:  0
shift_from:  -18
shift_to:  19
shift_step:  1
shift_indexes:  tensor([-18, -17, -16, -15, -14, -13, -12, -11, -10,  -9,  -8,  -7,  -6,  -5,
         -4,  -3,  -2,  -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,

[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7feef4708ac8>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7fef0460f550>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7fef0460f518>]
tensor(20, dtype=torch.int32)
dataset len:  498945
========== 1 generation ==========
temp_dataset_len:  0
shift_from:  -18
shift_to:  19
shift_step:  1
shift_indexes:  tensor([-18, -17, -16, -15, -14, -13, -12, -11, -10,  -9,  -8,  -7,  -6,  -5,
         -4,  -3,  -2,  -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
         10,  11,  12,  13,  14,  15,  16,  17,  18], dtype=torch.int32)
snr_ratio:  61.57137584737727
flip_data_horz:  0
[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7feef4708ac8>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7fef0460f550>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7fef04

dataset len:  494727
========== 1 generation ==========
temp_dataset_len:  0
shift_from:  -18
shift_to:  19
shift_step:  1
shift_indexes:  tensor([-18, -17, -16, -15, -14, -13, -12, -11, -10,  -9,  -8,  -7,  -6,  -5,
         -4,  -3,  -2,  -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
         10,  11,  12,  13,  14,  15,  16,  17,  18], dtype=torch.int32)
snr_ratio:  47.291155838758705
flip_data_horz:  0
[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7feef4708ac8>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7fef0460f550>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7fef0460f518>]
tensor(6, dtype=torch.int32)
dataset len:  233026
========== 1 generation ==========
temp_dataset_len:  0
shift_from:  -18
shift_to:  19
shift_step:  1
shift_indexes:  tensor([-18, -17, -16, -15, -14, -13, -12, -11, -10,  -9,  -8,  -7,  -6,  -5,
         -4,  -3,  -2,  -1,   0,   1,   2,   3,   4,   

[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7feef4708ac8>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7fef0460f550>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7fef0460f518>]
tensor(12, dtype=torch.int32)
dataset len:  350686
========== 1 generation ==========
temp_dataset_len:  0
shift_from:  -18
shift_to:  19
shift_step:  1
shift_indexes:  tensor([-18, -17, -16, -15, -14, -13, -12, -11, -10,  -9,  -8,  -7,  -6,  -5,
         -4,  -3,  -2,  -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
         10,  11,  12,  13,  14,  15,  16,  17,  18], dtype=torch.int32)
snr_ratio:  62.77317996384185
flip_data_horz:  0
[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7feef4708ac8>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7fef0460f550>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7fef04

In [11]:
unique_list = [];
dataset_noise = np.array([])
labels_noise = np.array([], dtype = "int")
    
# adds multi unit activity data
data_iter_counter = 0;
total_nb_of_samples = int(labels_spikes.size);
temp_spikes = np.zeros((total_nb_of_samples, 1, waveform_length));
temp_labels = np.zeros((1, total_nb_of_samples), dtype='int');
while (data_iter_counter < total_nb_of_samples):
    for i in range(0, nb_of_datasets):
        print(i)
        single_recording_path = path.join(path_to_recordings, 'data_' + str(i + 1) + '.npy');
        single_recording_ground_truth = path.join(path_to_ground_truth_data, 'gt_' + str(i + 1) + '.npy');
        noise_indices = cnn.GetNoiseIndices(single_recording_path, single_recording_ground_truth, waveform_length, nb_of_elements);
        noise_class = torch.zeros(1, noise_indices.nelement(), dtype= torch.int);
        noise_data = torch.cat((noise_indices, noise_class), 0);
        np.save(path_to_noise_data, noise_data.numpy());

        generated_spikes = cnn.GenerateDataset(single_recording_path, path_to_noise_data, waveform_length, 600, snr_from, snr_to, max_shift, use_horizontal_flip, use_vertical_flip, transform_list);
        # adds to temp variable to add to real dataset afterwards
        for j, (data, target) in enumerate(generated_spikes):
            neuron_index = target.item();
            if(neuron_index == 0 and data_iter_counter < total_nb_of_samples):
                spike_waveform =  (data.numpy());
                temp_spikes[data_iter_counter, :] = spike_waveform;
                class_index = neuron_index;
                # changes class index
                temp_labels[0, data_iter_counter] = class_index;
                data_iter_counter = data_iter_counter + 1;
                
        
        
unique_list.append(np.unique(temp_labels));
if (dataset_noise.size == 0):
    dataset_noise = temp_spikes;
else:
    dataset_noise = np.concatenate((dataset_noise, temp_spikes), axis=0)  
        
if (labels_noise.size == 0): 
    labels_noise = temp_labels;
else:
    labelspike_trains_noise = np.concatenate((labels_noise, temp_labels), axis=1)            

0
========== 1 generation ==========
temp_dataset_len:  0
shift_from:  -18
shift_to:  19
shift_step:  1
shift_indexes:  tensor([-18, -17, -16, -15, -14, -13, -12, -11, -10,  -9,  -8,  -7,  -6,  -5,
         -4,  -3,  -2,  -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
         10,  11,  12,  13,  14,  15,  16,  17,  18], dtype=torch.int32)
snr_ratio:  28.65533297254763
flip_data_horz:  0
[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7feef078cc50>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7fef0460f550>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7fef0460f518>]
tensor(0, dtype=torch.int32)
dataset len:  5550
1
========== 1 generation ==========
temp_dataset_len:  0
shift_from:  -18
shift_to:  19
shift_step:  1
shift_indexes:  tensor([-18, -17, -16, -15, -14, -13, -12, -11, -10,  -9,  -8,  -7,  -6,  -5,
         -4,  -3,  -2,  -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   

[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7feef078cc88>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7fef0460f550>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7fef0460f518>]
tensor(0, dtype=torch.int32)
dataset len:  5550
12
========== 1 generation ==========
temp_dataset_len:  0
shift_from:  -18
shift_to:  19
shift_step:  1
shift_indexes:  tensor([-18, -17, -16, -15, -14, -13, -12, -11, -10,  -9,  -8,  -7,  -6,  -5,
         -4,  -3,  -2,  -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
         10,  11,  12,  13,  14,  15,  16,  17,  18], dtype=torch.int32)
snr_ratio:  52.04732195400227
flip_data_horz:  0
[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7feef078ce80>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7fef0460f550>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7fef04

23
========== 1 generation ==========
temp_dataset_len:  0
shift_from:  -18
shift_to:  19
shift_step:  1
shift_indexes:  tensor([-18, -17, -16, -15, -14, -13, -12, -11, -10,  -9,  -8,  -7,  -6,  -5,
         -4,  -3,  -2,  -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
         10,  11,  12,  13,  14,  15,  16,  17,  18], dtype=torch.int32)
snr_ratio:  62.155741695070844
flip_data_horz:  0
[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7feef078cdd8>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7fef0460f550>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7fef0460f518>]
tensor(0, dtype=torch.int32)
dataset len:  5550
24
========== 1 generation ==========
temp_dataset_len:  0
shift_from:  -18
shift_to:  19
shift_step:  1
shift_indexes:  tensor([-18, -17, -16, -15, -14, -13, -12, -11, -10,  -9,  -8,  -7,  -6,  -5,
         -4,  -3,  -2,  -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,

[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7feef078cf98>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7fef0460f550>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7fef0460f518>]
tensor(0, dtype=torch.int32)
dataset len:  5550
35
========== 1 generation ==========
temp_dataset_len:  0
shift_from:  -18
shift_to:  19
shift_step:  1
shift_indexes:  tensor([-18, -17, -16, -15, -14, -13, -12, -11, -10,  -9,  -8,  -7,  -6,  -5,
         -4,  -3,  -2,  -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
         10,  11,  12,  13,  14,  15,  16,  17,  18], dtype=torch.int32)
snr_ratio:  71.1167368872065
flip_data_horz:  0
[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7feef078ce48>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7fef0460f550>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7fef046

1
========== 1 generation ==========
temp_dataset_len:  0
shift_from:  -18
shift_to:  19
shift_step:  1
shift_indexes:  tensor([-18, -17, -16, -15, -14, -13, -12, -11, -10,  -9,  -8,  -7,  -6,  -5,
         -4,  -3,  -2,  -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
         10,  11,  12,  13,  14,  15,  16,  17,  18], dtype=torch.int32)
snr_ratio:  89.25090932729455
flip_data_horz:  0
[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7feef07782b0>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7fef0460f550>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7fef0460f518>]
tensor(0, dtype=torch.int32)
dataset len:  5550
2
========== 1 generation ==========
temp_dataset_len:  0
shift_from:  -18
shift_to:  19
shift_step:  1
shift_indexes:  tensor([-18, -17, -16, -15, -14, -13, -12, -11, -10,  -9,  -8,  -7,  -6,  -5,
         -4,  -3,  -2,  -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   

...

[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7feef0778ef0>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7fef0460f550>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7fef0460f518>]
tensor(0, dtype=torch.int32)
dataset len:  5550
37
========== 1 generation ==========
temp_dataset_len:  0
shift_from:  -18
shift_to:  19
shift_step:  1
shift_indexes:  tensor([-18, -17, -16, -15, -14, -13, -12, -11, -10,  -9,  -8,  -7,  -6,  -5,
         -4,  -3,  -2,  -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
         10,  11,  12,  13,  14,  15,  16,  17,  18], dtype=torch.int32)
snr_ratio:  39.929180721767764
flip_data_horz:  0
[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7feef07789e8>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7fef0460f550>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7fef0

3
========== 1 generation ==========
temp_dataset_len:  0
shift_from:  -18
shift_to:  19
shift_step:  1
shift_indexes:  tensor([-18, -17, -16, -15, -14, -13, -12, -11, -10,  -9,  -8,  -7,  -6,  -5,
         -4,  -3,  -2,  -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
         10,  11,  12,  13,  14,  15,  16,  17,  18], dtype=torch.int32)
snr_ratio:  93.63330852064604
flip_data_horz:  0
[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7feef0778cc0>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7fef0460f550>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7fef0460f518>]
tensor(0, dtype=torch.int32)
dataset len:  5550
4
========== 1 generation ==========
temp_dataset_len:  0
shift_from:  -18
shift_to:  19
shift_step:  1
shift_indexes:  tensor([-18, -17, -16, -15, -14, -13, -12, -11, -10,  -9,  -8,  -7,  -6,  -5,
         -4,  -3,  -2,  -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   

[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7feef0778e10>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7fef0460f550>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7fef0460f518>]
tensor(0, dtype=torch.int32)
dataset len:  5550
15
========== 1 generation ==========
temp_dataset_len:  0
shift_from:  -18
shift_to:  19
shift_step:  1
shift_indexes:  tensor([-18, -17, -16, -15, -14, -13, -12, -11, -10,  -9,  -8,  -7,  -6,  -5,
         -4,  -3,  -2,  -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
         10,  11,  12,  13,  14,  15,  16,  17,  18], dtype=torch.int32)
snr_ratio:  100.19448738725903
flip_data_horz:  0
[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7feef0778dd8>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7fef0460f550>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7fef0

26
========== 1 generation ==========
temp_dataset_len:  0
shift_from:  -18
shift_to:  19
shift_step:  1
shift_indexes:  tensor([-18, -17, -16, -15, -14, -13, -12, -11, -10,  -9,  -8,  -7,  -6,  -5,
         -4,  -3,  -2,  -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
         10,  11,  12,  13,  14,  15,  16,  17,  18], dtype=torch.int32)
snr_ratio:  64.9566770958418
flip_data_horz:  0
[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7feef0778dd8>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7fef0460f550>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7fef0460f518>]
tensor(0, dtype=torch.int32)
dataset len:  5550
27
========== 1 generation ==========
temp_dataset_len:  0
shift_from:  -18
shift_to:  19
shift_step:  1
shift_indexes:  tensor([-18, -17, -16, -15, -14, -13, -12, -11, -10,  -9,  -8,  -7,  -6,  -5,
         -4,  -3,  -2,  -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,  

[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7feef0778fd0>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7fef0460f550>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7fef0460f518>]
tensor(0, dtype=torch.int32)
dataset len:  5550
38
========== 1 generation ==========
temp_dataset_len:  0
shift_from:  -18
shift_to:  19
shift_step:  1
shift_indexes:  tensor([-18, -17, -16, -15, -14, -13, -12, -11, -10,  -9,  -8,  -7,  -6,  -5,
         -4,  -3,  -2,  -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
         10,  11,  12,  13,  14,  15,  16,  17,  18], dtype=torch.int32)
snr_ratio:  29.88314454946126
flip_data_horz:  0
[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7feef0778dd8>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7fef0460f550>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7fef04

In [12]:
unique_list = [];
dataset_noise_multiunit = np.array([])
labels_noise_multiunit = np.array([], dtype = "int")
    
# adds multi unit activity data
data_iter_counter = 0;
total_nb_of_samples = int(labels_spikes.size);
temp_spikes = np.zeros((total_nb_of_samples, 1, waveform_length));
temp_labels = np.zeros((1, total_nb_of_samples), dtype='int');
while (data_iter_counter < total_nb_of_samples):
    for i in range(0, nb_of_datasets):
        single_recording_path = path.join(path_to_recordings, 'data_' + str(i + 1) + '.npy');
        single_recording_ground_truth = path.join(path_to_ground_truth_data, 'gt_' + str(i + 1) + '.npy');
        noise_indices = cnn.GetNoiseIndices(single_recording_path, single_recording_ground_truth, waveform_length, nb_of_elements);
        noise_class = torch.zeros(1, noise_indices.nelement(), dtype= torch.int);
        noise_data = torch.cat((noise_indices, noise_class), 0);
        np.save(path_to_noise_data, noise_data.numpy());

        generated_spikes = cnn.GenerateDataset(single_recording_path, path_to_noise_data, waveform_length, 600, snr_from, snr_to, max_shift, use_horizontal_flip, use_vertical_flip, transform_list);
        # adds to temp variable to add to real dataset afterwards
        for j, (data, target) in enumerate(generated_spikes):
            neuron_index = target.item();
            if(neuron_index == 0 and data_iter_counter < total_nb_of_samples):
                spike_waveform =  (data.numpy());
                temp_spikes[data_iter_counter, :] = spike_waveform;
                class_index = neuron_index;
                # changes class index
                temp_labels[0, data_iter_counter] = class_index;
                data_iter_counter = data_iter_counter + 1;
                
        
unique_list.append(np.unique(temp_labels));
if (dataset_noise_multiunit.size == 0):
    dataset_noise_multiunit = temp_spikes;
else:
    dataset_noise_multiunit = np.concatenate((dataset_noise_multiunit, temp_spikes), axis=0)  
        
if (labels_noise_multiunit.size == 0): 
    labels_noise_multiunit = temp_labels;
else:
    labels_noise_multiunit = np.concatenate((labels_noise_multiunit, temp_labels), axis=1)            

========== 1 generation ==========
temp_dataset_len:  0
shift_from:  -18
shift_to:  19
shift_step:  1
shift_indexes:  tensor([-18, -17, -16, -15, -14, -13, -12, -11, -10,  -9,  -8,  -7,  -6,  -5,
         -4,  -3,  -2,  -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
         10,  11,  12,  13,  14,  15,  16,  17,  18], dtype=torch.int32)
snr_ratio:  28.697317958037384
flip_data_horz:  0
[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7feef07788d0>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7fef0460f550>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7fef0460f518>]
tensor(0, dtype=torch.int32)
dataset len:  5550
========== 1 generation ==========
temp_dataset_len:  0
shift_from:  -18
shift_to:  19
shift_step:  1
shift_indexes:  tensor([-18, -17, -16, -15, -14, -13, -12, -11, -10,  -9,  -8,  -7,  -6,  -5,
         -4,  -3,  -2,  -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,


[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7feef0793588>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7fef0460f550>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7fef0460f518>]
tensor(0, dtype=torch.int32)
dataset len:  5550
========== 1 generation ==========
temp_dataset_len:  0
shift_from:  -18
shift_to:  19
shift_step:  1
shift_indexes:  tensor([-18, -17, -16, -15, -14, -13, -12, -11, -10,  -9,  -8,  -7,  -6,  -5,
         -4,  -3,  -2,  -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
         10,  11,  12,  13,  14,  15,  16,  17,  18], dtype=torch.int32)
snr_ratio:  59.76957476792497
flip_data_horz:  0
[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7feef0793780>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7fef0460f550>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7fef0460f

========== 1 generation ==========
temp_dataset_len:  0
shift_from:  -18
shift_to:  19
shift_step:  1
shift_indexes:  tensor([-18, -17, -16, -15, -14, -13, -12, -11, -10,  -9,  -8,  -7,  -6,  -5,
         -4,  -3,  -2,  -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
         10,  11,  12,  13,  14,  15,  16,  17,  18], dtype=torch.int32)
snr_ratio:  95.85753470443812
flip_data_horz:  0
[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7feef0793978>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7fef0460f550>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7fef0460f518>]
tensor(0, dtype=torch.int32)
dataset len:  5550
========== 1 generation ==========
temp_dataset_len:  0
shift_from:  -18
shift_to:  19
shift_step:  1
shift_indexes:  tensor([-18, -17, -16, -15, -14, -13, -12, -11, -10,  -9,  -8,  -7,  -6,  -5,
         -4,  -3,  -2,  -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
 

[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7feef0793320>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7fef0460f550>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7fef0460f518>]
tensor(0, dtype=torch.int32)
dataset len:  5550
========== 1 generation ==========
temp_dataset_len:  0
shift_from:  -18
shift_to:  19
shift_step:  1
shift_indexes:  tensor([-18, -17, -16, -15, -14, -13, -12, -11, -10,  -9,  -8,  -7,  -6,  -5,
         -4,  -3,  -2,  -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
         10,  11,  12,  13,  14,  15,  16,  17,  18], dtype=torch.int32)
snr_ratio:  31.604992393239847
flip_data_horz:  0
[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7feef0793320>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7fef0460f550>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7fef0460

========== 1 generation ==========
temp_dataset_len:  0
shift_from:  -18
shift_to:  19
shift_step:  1
shift_indexes:  tensor([-18, -17, -16, -15, -14, -13, -12, -11, -10,  -9,  -8,  -7,  -6,  -5,
         -4,  -3,  -2,  -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
         10,  11,  12,  13,  14,  15,  16,  17,  18], dtype=torch.int32)
snr_ratio:  28.16463603247603
flip_data_horz:  0
[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7feef0793438>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7fef0460f550>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7fef0460f518>]
tensor(0, dtype=torch.int32)
dataset len:  5550
========== 1 generation ==========
temp_dataset_len:  0
shift_from:  -18
shift_to:  19
shift_step:  1
shift_indexes:  tensor([-18, -17, -16, -15, -14, -13, -12, -11, -10,  -9,  -8,  -7,  -6,  -5,
         -4,  -3,  -2,  -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
 

...

[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7feef0795a20>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7fef0460f550>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7fef0460f518>]
tensor(0, dtype=torch.int32)
dataset len:  5550
========== 1 generation ==========
temp_dataset_len:  0
shift_from:  -18
shift_to:  19
shift_step:  1
shift_indexes:  tensor([-18, -17, -16, -15, -14, -13, -12, -11, -10,  -9,  -8,  -7,  -6,  -5,
         -4,  -3,  -2,  -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
         10,  11,  12,  13,  14,  15,  16,  17,  18], dtype=torch.int32)
snr_ratio:  55.11133190396656
flip_data_horz:  0
[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7feef0795780>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7fef0460f550>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7fef0460f

========== 1 generation ==========
temp_dataset_len:  0
shift_from:  -18
shift_to:  19
shift_step:  1
shift_indexes:  tensor([-18, -17, -16, -15, -14, -13, -12, -11, -10,  -9,  -8,  -7,  -6,  -5,
         -4,  -3,  -2,  -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
         10,  11,  12,  13,  14,  15,  16,  17,  18], dtype=torch.int32)
snr_ratio:  74.15966710690991
flip_data_horz:  0
[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7feef07957f0>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7fef0460f550>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7fef0460f518>]
tensor(0, dtype=torch.int32)
dataset len:  5550
========== 1 generation ==========
temp_dataset_len:  0
shift_from:  -18
shift_to:  19
shift_step:  1
shift_indexes:  tensor([-18, -17, -16, -15, -14, -13, -12, -11, -10,  -9,  -8,  -7,  -6,  -5,
         -4,  -3,  -2,  -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
 

[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7feef07959e8>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7fef0460f550>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7fef0460f518>]
tensor(0, dtype=torch.int32)
dataset len:  5550
========== 1 generation ==========
temp_dataset_len:  0
shift_from:  -18
shift_to:  19
shift_step:  1
shift_indexes:  tensor([-18, -17, -16, -15, -14, -13, -12, -11, -10,  -9,  -8,  -7,  -6,  -5,
         -4,  -3,  -2,  -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
         10,  11,  12,  13,  14,  15,  16,  17,  18], dtype=torch.int32)
snr_ratio:  71.10607692926997
flip_data_horz:  0
[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7feef0795c88>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7fef0460f550>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7fef0460f

========== 1 generation ==========
temp_dataset_len:  0
shift_from:  -18
shift_to:  19
shift_step:  1
shift_indexes:  tensor([-18, -17, -16, -15, -14, -13, -12, -11, -10,  -9,  -8,  -7,  -6,  -5,
         -4,  -3,  -2,  -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
         10,  11,  12,  13,  14,  15,  16,  17,  18], dtype=torch.int32)
snr_ratio:  50.565421055393216
flip_data_horz:  0
[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7feef0795e80>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7fef0460f550>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7fef0460f518>]
tensor(0, dtype=torch.int32)
dataset len:  5550
========== 1 generation ==========
temp_dataset_len:  0
shift_from:  -18
shift_to:  19
shift_step:  1
shift_indexes:  tensor([-18, -17, -16, -15, -14, -13, -12, -11, -10,  -9,  -8,  -7,  -6,  -5,
         -4,  -3,  -2,  -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,


[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7feef0795be0>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7fef0460f550>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7fef0460f518>]
tensor(0, dtype=torch.int32)
dataset len:  5550
========== 1 generation ==========
temp_dataset_len:  0
shift_from:  -18
shift_to:  19
shift_step:  1
shift_indexes:  tensor([-18, -17, -16, -15, -14, -13, -12, -11, -10,  -9,  -8,  -7,  -6,  -5,
         -4,  -3,  -2,  -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
         10,  11,  12,  13,  14,  15,  16,  17,  18], dtype=torch.int32)
snr_ratio:  86.52492677351724
flip_data_horz:  0
[ 0.98707844 -0.98707844] [ 1.         -0.97415687]
[<custom_resnet.CustomResnet.Awgn object at 0x7feef0795fd0>, <custom_resnet.CustomResnet.FilterSignalUsingButtersWorth object at 0x7fef0460f550>, <custom_resnet.CustomResnet.OptimizedZScoreNormalizaton object at 0x7fef0460f

In [13]:
dataset_spikes
print(labels_spikes.shape)
dataset_multiunit_spikes
print(labels_multiunit_spikes.shape)
dataset_noise
print(labels_noise.shape)
dataset_noise_multiunit
print(labels_noise_multiunit.shape)

(1, 998490)
(1, 998490)
(1, 998490)
(1, 998490)


In [27]:
labels_spikes1 = np.ones((labels_spikes.shape))
labels_multiunit_spikes1 = np.ones((labels_multiunit_spikes.shape))
labels_noise1 = np.zeros((labels_noise.shape))
labels_noise_multiunit1 = np.zeros((labels_noise_multiunit.shape))


In [28]:
labels_spikes1.shape

(1, 998490)

In [29]:
dataset1 = np.concatenate((dataset_spikes, dataset_noise), 0);
dataset2 = np.concatenate((dataset_multiunit_spikes, dataset_noise_multiunit), 0);
labels1 = np.concatenate((labels_spikes1, labels_noise1), 1); 
labels2 = np.concatenate((labels_multiunit_spikes1, labels_noise_multiunit1), 1); 

In [30]:
print(dataset1.shape)
print(dataset2.shape)
print(labels1.shape)
print(labels2.shape)

(1996980, 1, 72)
(1996980, 1, 72)
(1, 1996980)
(1, 1996980)


In [31]:
np.save(path_to_train_data, dataset1)
np.save(path_to_train_labels, labels1)
np.save(path_to_train_data1, dataset2)
np.save(path_to_train_labels1, labels2)

In [18]:
mean = (np.mean(np_data_spikes))
std = (np.std(np_data_spikes))
np.save(path_to_mean_std, [mean, std])

print(mean)
print(std)

NameError: name 'np_data_spikes' is not defined